In [1]:
import numpy as np
import pandas as pd
import sys
from hazm import *
from stopwords_guilannlp import *
from nltk.tokenize import RegexpTokenizer
from string import punctuation
import re
from collections import Counter
import math
import xlrd
import nltk
import math
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()


In [2]:
def load_datatset():
    dataset=pd.read_csv("ProjectData.csv")
    dataset=dataset[['comment','label']]
    dataset=dataset[dataset['label'] != -2]
    dataset=dataset.dropna()
    return dataset

In [3]:
def undersampl_data(dataset):
    counts = dataset['label'].value_counts()
    min_count = counts.min()
    undersampled_df = pd.concat([
        dataset[dataset['label'] == Label].sample(n=min_count)
        for Label in counts.index
    ])
    
    return undersampled_df

In [6]:
dataset=load_datatset()

In [5]:
dataset=undersampl_data(dataset)

In [7]:
counts = dataset ['label'].value_counts()

In [8]:
counts

 1.0    40149
-1.0    17044
 0.0     5627
Name: label, dtype: int64

In [9]:
dataset

,comment,label
0,گردن بند خوبو قشنگیه خوبم جلوه میکنه و خودشو ن...,1.0
1,به نظر من اصلا خوب نبود! به جاش با روغن زیتون ...,-1.0
2,من خریدم مبلم رو بی ریخت کرد و زود پاره شد,-1.0
3,حتما پیشنهاد میکنم,1.0
4,در کل عالی,1.0
...,...,...
62933,کواد کوپتر پرواز دادنش خیلی لذت بخش هست به شر...,1.0
62934,سلاممن ازش خیلس راضی هستم شارژ 10000 واقعی خیل...,1.0
62935,این فیلتر رو تا بحال دو بار نصب و تعویض کردم. ...,1.0
62936,ضد آفتاب مناسبی برای پوست چرب با رنگ‌خوب، البت...,1.0


In [10]:
def remove_stop_words(comments):
    stop_words = ['و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده_است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد_شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وارد', 'کل', 'ویژه', 'قبل', 'براساس', 'نیاز', 'گذاری', 'هنوز', 'لازم', 'سازی', 'بوده_است', 'چرا', 'می\u200cشوند', 'وقتی', 'گرفت', 'کم', 'جای', 'حالی', 'تغییر', 'پیدا', 'اکنون', 'تحت', 'باعث', 'مدت', 'فقط', 'تعداد', 'آیا', 'بیان', 'رو', 'شدند', 'عدم', 'کرده_اند', 'بودن', 'نوع', 'بلکه', 'جاری', 'دهد', 'برابر', 'مهم', 'بوده', 'اخیر', 'مربوط', 'امر', 'زیر', 'گیری', 'شاید', 'خصوص', 'آقای', 'اثر', 'کننده', 'بودند', 'فکر', 'کنار', 'اولین', 'سوم', 'سایر', 'کنید', 'ضمن', 'مانند', 'باز', 'می\u200cگیرد', 'ممکن', 'حل', 'دارای', 'پی', 'مثل', 'می\u200cرسد', 'اجرا', 'دور', 'منظور', 'کسی', 'موجب', 'طول', 'امکان', 'آنچه', 'تعیین', 'گفته', 'شوند', 'جمع', 'علاوه', 'گونه', 'تاکنون', 'رسید', 'ساله', 'گرفته', 'شده_اند', 'علت', 'چهار', 'داشته_باشد', 'خواهد_بود', 'طرف', 'تهیه', 'تبدیل', 'مناسب', 'زیرا', 'مشخص', 'می\u200cتوانند', 'نزدیک', 'جریان', 'روند', 'بنابراین', 'می\u200cدهند', 'یافت', 'نخستین', 'بالا', 'پنج', 'ریزی', 'چیزی', 'نخست', 'بیشتری', 'ترتیب', 'شده_بود', 'خاص', 'شروع', 'فرد', 'کامل', 'غیر', 'می\u200cرود', 'دهند', 'آخرین', 'دادن', 'جدی', 'بهترین', 'شامل', 'گیرد', 'بخشی', 'باشند', 'تمامی', 'بهتر', 'داده_است', 'حد', 'نبود', 'کسانی', 'می\u200cکرد', 'داریم', 'علیه', 'می\u200cباشد', 'دانست', 'ناشی', 'داشتند', 'دهه', 'می\u200cشد', 'ایشان', 'آنجا', 'گرفته_است', 'دچار', 'می\u200cآید', 'لحاظ', 'آنکه', 'داده', 'بعضی', 'هستیم', 'اند', 'برداری', 'نباید', 'می\u200cکنیم', 'نشست', 'سهم', 'همیشه', 'آمد', 'اش', 'وگو', 'می\u200cکنم', 'حداقل', 'طبق', 'جا', 'خواهد_کرد', 'نوعی', 'چگونه', 'رفت', 'هنگام', 'فوق', 'روش', 'ندارند', 'سعی', 'بندی', 'شمار', 'کلی', 'کافی', 'مواجه', 'همچنان', 'سمت', 'کوچک', 'داشته_است', 'چیز', 'پشت', 'آورد', 'حالا', 'روبه', 'سال\u200cهای', 'دادند', 'می\u200cکردند', 'عهده', 'نیمه', 'جایی', 'دیگران', 'سی', 'بروز', 'یکدیگر', 'آمده_است', 'جز', 'کنم', 'سپس', 'کنندگان', 'خودش', 'همواره', 'یافته', 'شان', 'صرف', 'نمی\u200cشود', 'رسیدن', 'چهارم', 'یابد', 'متر', 'ساز', 'داشته', 'کرده_بود', 'باره', 'نحوه', 'کردم', 'تو', 'شخصی', 'داشته_باشند', 'محسوب', 'پخش', 'کمی', 'متفاوت', 'سراسر', 'کاملا', 'داشتن', 'نظیر', 'آمده', 'گروهی', 'فردی', 'ع', 'همچون', 'خطر', 'خویش', 'کدام', 'دسته', 'سبب', 'عین', 'آوری', 'متاسفانه', 'بیرون', 'دار', 'ابتدا', 'شش', 'افرادی', 'می\u200cگویند', 'سالهای', 'درون', 'نیستند', 'یافته_است', 'پر', 'خاطرنشان', 'گاه', 'جمعی', 'اغلب', 'دوباره', 'می\u200cیابد', 'لذا', 'زاده', 'گردد', 'اینجا']
    REPLACE_NO_SPACE = re.compile("[.`;:!\'?,\"()\[\]،؛ًٌٍَُِّ]")
    REPLACE_NUMBER_ENGLISH = re.compile("[0-9A_Za-z۰-۹]")
    clean_comments = []
    comments = [REPLACE_NO_SPACE.sub("", line) for line in comments]
    comments = [REPLACE_NUMBER_ENGLISH.sub("", line) for line in comments]
    for review in comments:
        clean_comments.append(
            ' '.join([word for word in review.split() 
                      if word not in stop_words])
        )
    return clean_comments

In [11]:
def tokenize_text(text):
    text = normalizer.normalize(text)
    text = text.replace('.', ' ')
    text = re.sub('\s+', ' ', text).strip()
    text = text.replace('\u200c', ' ').replace('\n', '').replace('\r', '').replace('ي', 'ی').replace('ك', 'ک')
    tokens = word_tokenize(text)
    return tokens


In [12]:
def create_word_set(comments):
    word_set = set()
    for comment in comments:
        for token in comment:
            word_set.add(token)
    return word_set

In [13]:
def preprocessing(comment):
    stop_words = ['و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده_است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد_شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وارد', 'کل', 'ویژه', 'قبل', 'براساس', 'نیاز', 'گذاری', 'هنوز', 'لازم', 'سازی', 'بوده_است', 'چرا', 'می\u200cشوند', 'وقتی', 'گرفت', 'کم', 'جای', 'حالی', 'تغییر', 'پیدا', 'اکنون', 'تحت', 'باعث', 'مدت', 'فقط', 'تعداد', 'آیا', 'بیان', 'رو', 'شدند', 'عدم', 'کرده_اند', 'بودن', 'نوع', 'بلکه', 'جاری', 'دهد', 'برابر', 'مهم', 'بوده', 'اخیر', 'مربوط', 'امر', 'زیر', 'گیری', 'شاید', 'خصوص', 'آقای', 'اثر', 'کننده', 'بودند', 'فکر', 'کنار', 'اولین', 'سوم', 'سایر', 'کنید', 'ضمن', 'مانند', 'باز', 'می\u200cگیرد', 'ممکن', 'حل', 'دارای', 'پی', 'مثل', 'می\u200cرسد', 'اجرا', 'دور', 'منظور', 'کسی', 'موجب', 'طول', 'امکان', 'آنچه', 'تعیین', 'گفته', 'شوند', 'جمع', 'علاوه', 'گونه', 'تاکنون', 'رسید', 'ساله', 'گرفته', 'شده_اند', 'علت', 'چهار', 'داشته_باشد', 'خواهد_بود', 'طرف', 'تهیه', 'تبدیل', 'مناسب', 'زیرا', 'مشخص', 'می\u200cتوانند', 'نزدیک', 'جریان', 'روند', 'بنابراین', 'می\u200cدهند', 'یافت', 'نخستین', 'بالا', 'پنج', 'ریزی', 'چیزی', 'نخست', 'بیشتری', 'ترتیب', 'شده_بود', 'خاص', 'شروع', 'فرد', 'کامل', 'غیر', 'می\u200cرود', 'دهند', 'آخرین', 'دادن', 'جدی', 'بهترین', 'شامل', 'گیرد', 'بخشی', 'باشند', 'تمامی', 'بهتر', 'داده_است', 'حد', 'نبود', 'کسانی', 'می\u200cکرد', 'داریم', 'علیه', 'می\u200cباشد', 'دانست', 'ناشی', 'داشتند', 'دهه', 'می\u200cشد', 'ایشان', 'آنجا', 'گرفته_است', 'دچار', 'می\u200cآید', 'لحاظ', 'آنکه', 'داده', 'بعضی', 'هستیم', 'اند', 'برداری', 'نباید', 'می\u200cکنیم', 'نشست', 'سهم', 'همیشه', 'آمد', 'اش', 'وگو', 'می\u200cکنم', 'حداقل', 'طبق', 'جا', 'خواهد_کرد', 'نوعی', 'چگونه', 'رفت', 'هنگام', 'فوق', 'روش', 'ندارند', 'سعی', 'بندی', 'شمار', 'کلی', 'کافی', 'مواجه', 'همچنان', 'سمت', 'کوچک', 'داشته_است', 'چیز', 'پشت', 'آورد', 'حالا', 'روبه', 'سال\u200cهای', 'دادند', 'می\u200cکردند', 'عهده', 'نیمه', 'جایی', 'دیگران', 'سی', 'بروز', 'یکدیگر', 'آمده_است', 'جز', 'کنم', 'سپس', 'کنندگان', 'خودش', 'همواره', 'یافته', 'شان', 'صرف', 'نمی\u200cشود', 'رسیدن', 'چهارم', 'یابد', 'متر', 'ساز', 'داشته', 'کرده_بود', 'باره', 'نحوه', 'کردم', 'تو', 'شخصی', 'داشته_باشند', 'محسوب', 'پخش', 'کمی', 'متفاوت', 'سراسر', 'کاملا', 'داشتن', 'نظیر', 'آمده', 'گروهی', 'فردی', 'ع', 'همچون', 'خطر', 'خویش', 'کدام', 'دسته', 'سبب', 'عین', 'آوری', 'متاسفانه', 'بیرون', 'دار', 'ابتدا', 'شش', 'افرادی', 'می\u200cگویند', 'سالهای', 'درون', 'نیستند', 'یافته_است', 'پر', 'خاطرنشان', 'گاه', 'جمعی', 'اغلب', 'دوباره', 'می\u200cیابد', 'لذا', 'زاده', 'گردد', 'اینجا']
    REPLACE_NO_SPACE = re.compile("[.`;:!\'?,\"()\[\]،؛ًٌٍَُِّ]")
    REPLACE_NUMBER_ENGLISH = re.compile("[0-9A_Za-z۰-۹]")
    comment = [REPLACE_NO_SPACE.sub("", line) for line in comment]
    comment = [REPLACE_NUMBER_ENGLISH.sub("", line) for line in comment]
    comment = ''.join(c for c in comment if not c.isdigit())
    comment = ''.join(c for c in comment if c not in punctuation)
    comment = normalizer.normalize(comment)
    tokens = word_tokenize(comment)
    cleared_text = []
    for word in tokens:
        word = normalizer.normalize(word)
#         word = stemmer.stem(word)
        word = lemmatizer.lemmatize(word)
          #word = re.sub(r'\d+', '', word)
        if word not in stop_words and len(word) > 1:
                cleared_text.append(word)
    return cleared_text

In [14]:
X=(dataset.iloc[:, :-1].values)
y=dataset.iloc[:, -1]

In [15]:
tokens=[]
for i in range(len(X)):
    tokens.append(preprocessing((X[i])))

In [16]:
def DF(tokens):
    word_counts = Counter(word for feature in tokens for word in set(feature))
    total_comments = len(tokens)
    upper_threshold = total_comments * 0.8
    lower_threshold = total_comments * 0.001
    pruned_tokens_features = []
    for feature in tokens:
        pruned_feature = [word for word in feature if word_counts[word] < upper_threshold and word_counts[word] >= lower_threshold]
        pruned_tokens_features.append(pruned_feature)

    return pruned_tokens_features

In [17]:
temp=DF(tokens)


In [18]:
tokens=temp

In [19]:
def cal_Tf_Idf(tokens):
    tf = []
    for doc in tokens:
        doc_tf = {}
        for word in doc:
            doc_tf[word] = doc.count(word) / len(doc)
        tf.append(doc_tf)
    idf = {}
    for doc in tokens:
        for word in set(doc):
            if word in idf:
                idf[word] += 1
            else:
                idf[word] = 1

    num_docs = len(tokens)
    for word in idf:
        idf[word] = math.log((1 + num_docs) / (1 + idf[word])) + 1
        
    tfidf = []
    for doc in tf:
        doc_tfidf = {}
        for word in doc:
            doc_tfidf[word] = doc[word] * idf[word]
        tfidf.append(doc_tfidf)

    # Normalize the TF-IDF score for each word in each document
    for i in range(len(tfidf)):
        tfidf_values = list(tfidf[i].values())
        norm = math.sqrt(sum(x**2 for x in tfidf_values))
        for word in tfidf[i]:
            tfidf[i][word] /= norm
            
    vocab = sorted(set(word for doc in tokens for word in doc))
    matrix = [[doc.get(word, 0) for word in vocab] for doc in tfidf]
    matrix = np.array(matrix)
    return matrix

In [61]:
matrix=cal_Tf_Idf(tokens)

In [62]:
matrix.shape

(62820, 1782)

In [63]:
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2, k=00).fit_transform(matrix,y)


In [50]:
matrix=X_pca

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size=0.2, random_state=42)


In [65]:

# Build the logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Accuracy: 0.7860553963705826


C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)



array([[2419,   17,  913],
       [ 328,   39,  778],
       [ 542,   34, 7494]], dtype=int64)

In [66]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test the classifier
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7425978987583572


In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[2419,   17,  913],
       [ 328,   39,  778],
       [ 542,   34, 7494]], dtype=int64)

In [67]:
from sklearn import svm
clf = svm.SVC(kernel='linear', decision_function_shape='ovr') # OVR stands for One-vs-Rest
clf.fit(X_train, y_train)

# Predict the test set labels
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7827921044253422


In [204]:
a=create_word_set(temp)
len(a)

1782